In [ ]:
# Interpola las variables y la máscara con
# el método "nearest" para poder tener
# información en todos los municipios.

import os
import rioxarray

import pandas as pd
import numpy as np

import xarray as xr
import geopandas as gpd

In [ ]:
path_drive = "/content/drive/MyDrive/Colab/"

path_d = [path_drive + "CHIRTS/",
    path_drive + "CHIRTS/",
    path_drive + "CHIRPS/"]
path_mask = path_drive + "municipios_mask.nc"
path_shp = path_drive + "Municipios/mun20gw.shp"

vars = ["Tmax", "Tmin", "Pre"]

# Si no existen la carpetas, las crea.
if not os.path.exists(path_drive + "vars/"):
    os.mkdir(path_drive + "vars/")
    for i in vars:
        os.mkdir(path_drive + "vars/" + i)

In [ ]:
# Nuevo rango de latitud y longitud
lat = np.linspace(14.574999, 32.724999, 727)
lon = np.linspace(-118.374999, -86.674999, 1269)

# Interpolación de las variables.
for i in range(len(vars)):
    for j in range(1995, 2017):
        ds = ( xr.open_dataset(path_d[i]
            + vars[i] + "/mexico_" + vars[i]
            + "." + str(j) + ".nc") )
        ds = ds.where(ds > -9000)
        ds_int = ( ds.interp(lat = lat,
            lon = lon, method = "nearest") )
        ds_int.to_netcdf(path_drive + "vars/" 
            + vars[i] + "/mexico_" + vars[i]
            + "." + str(j) + ".nc")

In [ ]:
# Cargamos el archivo base para las máscaras.
ds = xr.open_dataset(path_drive + "vars/"
    + vars[i] + "/mexico_" + vars[i]
    + ".1995.nc")

# Se cargan los municipios.
gdf = gpd.read_file( path_shp )

# Se obtiene el contorno de los municiipios.
gdf["boundary"] = gdf.boundary

# Se establece el datum de los datos.
ds = ds.rio.write_crs(gdf.crs)

# Se hace un Dataset con unos para todo el país
mask_0 = xr.ones_like( ds.isel(time = 0)
    .drop(["time", "spatial_ref"])
    .expand_dims(dim = "municipio"),
    dtype = int ).rio.write_crs(gdf.crs)

# La lista que contra los Datasets la 
# máscara para cada municipio.
mask = []

# Se hace la máscara para cada municipio.
for i in range(gdf.shape[0]):
    mask.append( mask_0.rio.clip(
        gdf[gdf.index == i].geometry,
        gdf.crs, drop=False, invert=False) )

# Se unen las máscaras para todos los municipios.
mask = ( xr.concat(mask, dim = "municipio")
    .drop("spatial_ref") )
mask = mask.where(mask > -9000)

In [ ]:
# Guardamo☼ la máscara
mask.to_netcdf(path_drive + "municipios_mask_025.nc")